# Data prepration

In [1]:
import numpy as np
import datetime
import os
import sklearn.metrics
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from scipy.stats import chi
from scipy import stats
from functions import mean_gen, varx_gen, vary_gen, spherical_data, discrete_angles

2022-08-10 14:57:00.214560: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-10 14:57:00.214593: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
n = 500000


m_th = 50
m_ph = 50
thetas = discrete_angles(n, m_th, 2)
phis = discrete_angles(n, m_ph, 1)
name = ('discrete_model_mth%.0f_mph%.0f' %(m_th, m_ph))


# angles = np.array([5, 3])
# thetas, phis = angles[0] * np.ones(n), angles[1] * np.ones(n)
# name = ('discrete_model_th%.0f_ph%.0f' %(angles[0], angles[1]))

x_train, x_test, y_train, y_test = spherical_data(n, thetas, phis)

print(x_train)

[[ 0.0310779  -0.50671447  2.65830753  2.45370394]
 [ 0.11529899 -0.80340098  2.18677267  0.56888261]
 [ 1.64012867  0.26426903  1.58085041  0.03744665]
 ...
 [-0.16940264  1.0686722   1.58085041  0.03744665]
 [ 0.31660958  1.18186454  0.5272643   3.1315843 ]
 [ 1.11525555  0.40472511  3.6402614   1.22259782]]


# Training Interpolated Model

In [3]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
]);

loss_fn = tf.keras.losses.BinaryCrossentropy()
model.compile(
    optimizer='adam',
    loss=loss_fn,
    metrics=['accuracy']
)

batch_size = 5000

checkpoint_path = "tutorial/checkpoints/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    save_weights_only=True,
    verbose=1,
    save_freq=10*batch_size
)

log_dir = "tutorial/logs/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

EPOCHS=30
model.fit(
    x_train, y_train, epochs=EPOCHS,
    batch_size=batch_size,
    callbacks=[cp_callback, tensorboard_callback]
);

model.save('3dmodels/' + name)

2022-08-10 14:57:05.942552: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-08-10 14:57:05.942600: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-08-10 14:57:05.942625: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jupyter-matan): /proc/driver/nvidia/version does not exist
2022-08-10 14:57:05.942969: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-10 14:57:05.976610: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initiali

Epoch 1/30
  6/160 [>.............................] - ETA: 5s - loss: 0.6771 - accuracy: 0.5664

2022-08-10 14:57:06.971287: I tensorflow/core/profiler/lib/profiler_session.cc:131] Profiler session initializing.
2022-08-10 14:57:06.971323: I tensorflow/core/profiler/lib/profiler_session.cc:146] Profiler session started.
2022-08-10 14:57:06.994861: I tensorflow/core/profiler/lib/profiler_session.cc:66] Profiler session collecting data.
2022-08-10 14:57:06.996646: I tensorflow/core/profiler/lib/profiler_session.cc:164] Profiler session tear down.
2022-08-10 14:57:07.011413: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: tutorial/logs/20220810-145705/train/plugins/profile/2022_08_10_14_57_07

2022-08-10 14:57:07.017823: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to tutorial/logs/20220810-145705/train/plugins/profile/2022_08_10_14_57_07/jupyter-matan.trace.json.gz
2022-08-10 14:57:07.022686: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: tutorial/logs/20220810-145

160/160 [==============================] - 3s 17ms/step - loss: 0.6476 - accuracy: 0.6031
Epoch 2/30
160/160 [==============================] - 3s 16ms/step - loss: 0.6399 - accuracy: 0.6122
Epoch 3/30
160/160 [==============================] - 2s 13ms/step - loss: 0.6368 - accuracy: 0.6179
Epoch 4/30
160/160 [==============================] - 2s 13ms/step - loss: 0.6347 - accuracy: 0.6221
Epoch 5/30
160/160 [==============================] - 2s 14ms/step - loss: 0.6332 - accuracy: 0.6241
Epoch 6/30
160/160 [==============================] - 2s 15ms/step - loss: 0.6318 - accuracy: 0.6269
Epoch 7/30
160/160 [==============================] - 2s 15ms/step - loss: 0.6311 - accuracy: 0.6274
Epoch 8/30
160/160 [==============================] - 2s 14ms/step - loss: 0.6305 - accuracy: 0.6279
Epoch 9/30
160/160 [==============================] - 3s 16ms/step - loss: 0.6298 - accuracy: 0.6278
Epoch 10/30
160/160 [==============================] - 2s 15ms/step - loss: 0.6292 - accuracy: 0.6285


2022-08-10 14:58:13.063073: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: 3dmodels/discrete_model_mth50_mph50/assets


In [16]:
model = tf.keras.models.load_model('3dmodels/' + name)